In [1]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

# from PIL import Image
from utils import *


# load data

In [2]:
combined_all_dataframes = get_postProcessedData()
df_feat = getFixationFeaturesDf(combined_all_dataframes,remove_data_after_spot_c  =True)

index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 35
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 102
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 18
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 84


# Logistic regression

In [7]:
df_feat.Total_Fixation_time_in_mask.mean()

592.4996282467532

In [4]:
df_feat.head()

,Participant,Image_n,Seq_Trial_N,Trial_Start,Response_time,Total_Fixation_count,Count_Fixations_in_mask,Total_Fixation_time_in_mask,Total_Fixation_time_out_mask,Time_of_first_Fixation_in_mask,Min_Dist_to_mask_center,Max_Dist_to_mask_center,Avg_Dist_to_mask_center,Clicked_on_mask,fixation_in_mask_both,Fixation_in_Mask,Mouse_in_Mask
0,Participant1,118,2,Present,NaN,36,0,0.000,10705.729,NaN,113.348515,470.105050,251.235201,0,0,0,False
1,Participant1,5,3,Present,9305029.0,34,8,2913.738,6161.608,7594559.0,3.821404,730.211619,273.471834,1,1,1,True
2,Participant1,116,4,Present,NaN,46,0,0.000,10035.142,NaN,79.408721,779.660393,399.347449,0,0,0,False
3,Participant1,61,5,Present,NaN,41,2,399.712,10087.398,571902.0,32.975438,526.802890,253.561892,0,1,1,False
4,Participant1,115,6,Present,12748102.0,38,5,1186.474,8768.309,11099876.0,16.441238,738.105504,308.287360,1,0,1,True


In [13]:
df_feat.head()

,Participant,Image_n,Seq_Trial_N,Trial_Start,Response_time,Total_Fixation_count,Count_Fixations_in_mask,Total_Fixation_time_in_mask,Total_Fixation_time_out_mask,Time_of_first_Fixation_in_mask,Min_Dist_to_mask_center,Max_Dist_to_mask_center,Avg_Dist_to_mask_center,Clicked_on_mask,fixation_in_mask_both,Fixation_in_Mask,Mouse_in_Mask,Intercept
0,Participant1,118,2,Present,NaN,36,0,0.000000,10.705729,NaN,113.348515,470.105050,251.235201,0,0,0,False,1
1,Participant1,5,3,Present,9305029.0,34,8,2.913738,6.161608,7594559.0,3.821404,730.211619,273.471834,1,1,1,True,1
2,Participant1,116,4,Present,NaN,46,0,0.000000,10.035142,NaN,79.408721,779.660393,399.347449,0,0,0,False,1
3,Participant1,61,5,Present,NaN,41,2,0.399712,10.087398,571902.0,32.975438,526.802890,253.561892,0,1,1,False,1
4,Participant1,115,6,Present,12748102.0,38,5,1.186474,8.768309,11099876.0,16.441238,738.105504,308.287360,1,0,1,True,1


In [8]:
# change fixation in mask to -1 and 1
# df_feat['fixation_in_mask_both'] = df_feat['fixation_in_mask_both'].replace(0,-1)
# df.replace(0, 5)

In [10]:
# # add constant intercept to data
# df_feat['Intercept'] = 1.0

In [11]:
# add constant intercept to data
df_feat['Intercept'] = 1.0

# add interaction time and both
df_feat['interaction'] = df_feat.Total_Fixation_time_in_mask * df_feat.fixation_in_mask_both

In [54]:
# df_feat.fixation_in_mask_both


0     -1
1      1
2     -1
3      1
4     -1
      ..
611    1
612    1
613    1
614   -1
615    1
Name: fixation_in_mask_both, Length: 616, dtype: int32

In [22]:
df_feat.columns

Index(['Participant', 'Image_n', 'Seq_Trial_N', 'Trial_Start', 'Response_time',
       'Total_Fixation_count', 'Count_Fixations_in_mask',
       'Total_Fixation_time_in_mask', 'Total_Fixation_time_out_mask',
       'Time_of_first_Fixation_in_mask', 'Min_Dist_to_mask_center',
       'Max_Dist_to_mask_center', 'Avg_Dist_to_mask_center', 'Clicked_on_mask',
       'fixation_in_mask_both', 'Fixation_in_Mask', 'Mouse_in_Mask',
       'Intercept', 'interaction'],
      dtype='object')

In [12]:
Xtrain = df_feat[[
    # 'Fixation_in_Mask',
    'fixation_in_mask_both',
    'Total_Fixation_time_in_mask',
    # 'interaction',
    # 'Count_Fixations_in_mask',
    # 'Min_Dist_to_mask_center',
    'Intercept'
    ]]
# Xtrain = df_feat[[
#     'Min_Dist_to_mask_center',
#     'Total_Fixation_time_in_mask'
#     # 'Fixation_in_Mask'
#     ]]

# ytrain = df_feat[['Clicked_on_mask']]
ytrain = df_feat[['Mouse_in_Mask']].astype(int)    

# # building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit(method ='bfgs')
print(log_reg.summary())


Optimization terminated successfully.
         Current function value: 0.474592
         Iterations: 18
         Function evaluations: 27
         Gradient evaluations: 21
                           Logit Regression Results                           
Dep. Variable:          Mouse_in_Mask   No. Observations:                  616
Model:                          Logit   Df Residuals:                      613
Method:                           MLE   Df Model:                            2
Date:                Tue, 13 Dec 2022   Pseudo R-squ.:                  0.2979
Time:                        14:40:26   Log-Likelihood:                -292.35
converged:                       True   LL-Null:                       -416.37
Covariance Type:            nonrobust   LLR p-value:                 1.376e-54
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
fixa

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [26]:
Xtrain = df_feat[[
    # 'Fixation_in_Mask',
    'fixation_in_mask_both',
    'Total_Fixation_time_in_mask',
    'Count_Fixations_in_mask',
    'Min_Dist_to_mask_center',
    # 'Avg_Dist_to_mask_center',
    'Intercept'
    ]]
# Xtrain = df_feat[[
#     'Min_Dist_to_mask_center',
#     'Total_Fixation_time_in_mask'
#     # 'Fixation_in_Mask'
#     ]]

# ytrain = df_feat[['Clicked_on_mask']]
ytrain = df_feat[['Mouse_in_Mask']].astype(int)    

# # building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit(method ='bfgs')
print(log_reg.summary())


Optimization terminated successfully.
         Current function value: 0.443331
         Iterations: 27
         Function evaluations: 36
         Gradient evaluations: 30
                           Logit Regression Results                           
Dep. Variable:          Mouse_in_Mask   No. Observations:                  616
Model:                          Logit   Df Residuals:                      611
Method:                           MLE   Df Model:                            4
Date:                Tue, 13 Dec 2022   Pseudo R-squ.:                  0.3441
Time:                        14:52:30   Log-Likelihood:                -273.09
converged:                       True   LL-Null:                       -416.37
Covariance Type:            nonrobust   LLR p-value:                 8.606e-61
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
fixa

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [14]:
# # predict values
# Xtest = [1,3]
# yhat = log_reg.predict(Xtest)


In [26]:
ytrain

,Mouse_in_Mask
0,0
1,1
2,0
3,0
4,1
...,...
611,0
612,0
613,1
614,0


# statistical analysis with updated features

In [3]:
df_feat = calculate_features_from_df(combined_all_dataframes, remove_example_trials = True,remove_data_after_spot_c  = True )


index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 35
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 102
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 18
index 0 is out of bounds for axis 0 with size 0
WARNING, failed to caluclate for Participant: Participant15 on Image: 84


In [4]:
df_feat.head()

,Participant,Image_n,Seq_Trial_N,Trial_Start,Response_time,Total_Fixation_count,Count_Fixations_in_mask,Total_Fixation_time_in_mask,Total_Fixation_time_out_mask,Time_of_first_Fixation_in_mask,Fixate_in_both_conditions,Fixate_in_any_conditions,Fixated_once_in_mask,Min_Dist_to_mask_center,Max_Dist_to_mask_center,Avg_Dist_to_mask_center,Clicked_on_mask
0,Participant1,118,2,Present,NaN,36,0,0.000,10705.729,NaN,0,0,0,113.348515,470.105050,251.235201,0
1,Participant1,5,3,Present,9305029.0,28,3,913.470,5945.097,7594559.0,1,0,1,6.201546,730.211619,308.380428,1
2,Participant1,116,4,Present,NaN,46,0,0.000,10035.142,NaN,0,0,0,79.408721,779.660393,399.347449,0
3,Participant1,61,5,Present,NaN,41,2,399.712,10087.398,571902.0,0,1,1,32.975438,526.802890,253.561892,0
4,Participant1,115,6,Present,12748102.0,37,4,1168.177,8768.309,11099876.0,1,0,1,16.441238,738.105504,315.902228,1


In [16]:
df_feat.head()

,Participant,Image_n,Seq_Trial_N,Trial_Start,Response_time,Total_Fixation_count,Count_Fixations_in_mask,Total_Fixation_time_in_mask,Total_Fixation_time_out_mask,Time_of_first_Fixation_in_mask,Fixate_in_both_conditions,Fixate_in_any_conditions,Fixated_once_in_mask,Min_Dist_to_mask_center,Max_Dist_to_mask_center,Avg_Dist_to_mask_center,Clicked_on_mask
0,Participant1,118,2,Present,NaN,36,0,0.000,10705.729,NaN,0,0,0,113.348515,470.105050,251.235201,0
1,Participant1,5,3,Present,9305029.0,28,3,913.470,5945.097,7594559.0,1,0,1,6.201546,730.211619,308.380428,1
2,Participant1,116,4,Present,NaN,46,0,0.000,10035.142,NaN,0,0,0,79.408721,779.660393,399.347449,0
3,Participant1,61,5,Present,NaN,41,2,399.712,10087.398,571902.0,0,1,1,32.975438,526.802890,253.561892,0
4,Participant1,115,6,Present,12748102.0,37,4,1168.177,8768.309,11099876.0,1,0,1,16.441238,738.105504,315.902228,1


In [18]:
df_feat[(df_feat.Fixated_once_in_mask ==1) & (df_feat.Fixate_in_any_conditions ==0)]

,Participant,Image_n,Seq_Trial_N,Trial_Start,Response_time,Total_Fixation_count,Count_Fixations_in_mask,Total_Fixation_time_in_mask,Total_Fixation_time_out_mask,Time_of_first_Fixation_in_mask,Fixate_in_both_conditions,Fixate_in_any_conditions,Fixated_once_in_mask,Min_Dist_to_mask_center,Max_Dist_to_mask_center,Avg_Dist_to_mask_center,Clicked_on_mask
1,Participant1,5,3,Present,9305029.0,28,3,913.470,5945.097,7594559.0,1,0,1,6.201546,730.211619,308.380428,1
4,Participant1,115,6,Present,12748102.0,37,4,1168.177,8768.309,11099876.0,1,0,1,16.441238,738.105504,315.902228,1
8,Participant1,97,10,Absent,5768082.0,19,4,821.733,3337.970,1118727.0,1,0,1,9.110568,461.941139,186.116019,1
9,Participant1,22,11,Absent,3226367.0,13,0,0.000,2153.766,NaN,1,0,1,122.769311,456.866757,259.412607,1
10,Participant1,64,12,Absent,NaN,40,3,843.320,9741.066,694996.0,1,0,1,40.743406,735.153604,346.148156,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Participant22,120,2,Present,5838653.0,13,3,1165.831,2203.288,857220.0,1,0,1,31.365122,392.598264,175.813107,1
602,Participant22,60,24,Absent,NaN,39,7,1855.458,7677.476,3583323.0,1,0,1,16.456453,454.278599,264.611189,0
610,Participant22,49,21,Present,3258082.0,11,2,509.577,1656.978,2033320.0,1,0,1,8.863803,716.957454,340.116274,1
613,Participant22,40,20,Present,7997895.0,24,2,764.372,4304.624,6281903.0,1,0,1,18.583456,717.847319,408.435411,1


# logistic regression 2.o

In [6]:
# add constant intercept to data
df_feat['Intercept'] = 1.0

# add interaction time and both
df_feat['interaction'] = df_feat.Total_Fixation_time_in_mask * df_feat.Fixate_in_both_conditions

In [24]:
Xtrain = df_feat[[
    # 'Fixated_once_in_mask',
    'Fixate_in_both_conditions',
    'Fixate_in_any_conditions',
    'Total_Fixation_time_in_mask',
    'Count_Fixations_in_mask',
    # 'Min_Dist_to_mask_center',
    # 'Avg_Dist_to_mask_center',
    # 'interaction'
    'Intercept'
    ]]
# Xtrain = df_feat[[
#     'Min_Dist_to_mask_center',
#     'Total_Fixation_time_in_mask'
#     # 'Fixation_in_Mask'
#     ]]

# ytrain = df_feat[['Clicked_on_mask']]
ytrain = df_feat[['Clicked_on_mask']].astype(int)    

# # building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit(method ='bfgs')
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.342543
         Iterations: 33
         Function evaluations: 42
         Gradient evaluations: 36
                           Logit Regression Results                           
Dep. Variable:        Clicked_on_mask   No. Observations:                  616
Model:                          Logit   Df Residuals:                      611
Method:                           MLE   Df Model:                            4
Date:                Tue, 13 Dec 2022   Pseudo R-squ.:                  0.4937
Time:                        20:25:12   Log-Likelihood:                -211.01
converged:                       True   LL-Null:                       -416.74
Covariance Type:            nonrobust   LLR p-value:                 9.258e-88
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Fixa

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


# get number of cases clicked and not

In [19]:
df_feat.columns

Index(['Participant', 'Image_n', 'Seq_Trial_N', 'Trial_Start', 'Response_time',
       'Total_Fixation_count', 'Count_Fixations_in_mask',
       'Total_Fixation_time_in_mask', 'Total_Fixation_time_out_mask',
       'Time_of_first_Fixation_in_mask', 'Fixate_in_both_conditions',
       'Fixate_in_any_conditions', 'Fixated_once_in_mask',
       'Min_Dist_to_mask_center', 'Max_Dist_to_mask_center',
       'Avg_Dist_to_mask_center', 'Clicked_on_mask', 'Intercept',
       'interaction'],
      dtype='object')

In [27]:
df_feat.groupby(by = 'Fixated_once_in_mask')[['Clicked_on_mask']].sum()

,Clicked_on_mask
Fixated_once_in_mask,
0,4
1,248


In [32]:
varlist = ['Fixate_in_both_conditions','Fixated_once_in_mask','Fixate_in_any_conditions']
for var in varlist:
    print(df_feat.groupby(by = var)[['Clicked_on_mask']].sum())
    # print(df_feat.groupby(by = var)[['Clicked_on_mask']].count_values(0))

                           Clicked_on_mask
Fixate_in_both_conditions                 
0                                       25
1                                      227
                      Clicked_on_mask
Fixated_once_in_mask                 
0                                   4
1                                 248
                          Clicked_on_mask
Fixate_in_any_conditions                 
0                                     231
1                                      21


In [34]:
df_feat.groupby(by = 'Clicked_on_mask')[varlist].sum()

,Fixate_in_both_conditions,Fixated_once_in_mask,Fixate_in_any_conditions
Clicked_on_mask,,,
0,64,208,144
1,227,248,21


In [36]:
meanvarlist = ['Total_Fixation_time_in_mask','Count_Fixations_in_mask']
df_feat.groupby(by = 'Clicked_on_mask')[meanvarlist].mean()

,Total_Fixation_time_in_mask,Count_Fixations_in_mask
Clicked_on_mask,,
0,262.680690,1.217033
1,1068.904762,2.932540


In [40]:
# didnt fixate mask
print(df_feat[(df_feat.Clicked_on_mask == 1) & (df_feat.Fixated_once_in_mask == 0)].shape)
print(df_feat[(df_feat.Clicked_on_mask == 0) & (df_feat.Fixated_once_in_mask == 0)].shape)


(4, 19)
(156, 19)


In [41]:
print(df_feat[(df_feat.Clicked_on_mask == 1) & (df_feat.Fixate_in_both_conditions == 0)].shape)
print(df_feat[(df_feat.Clicked_on_mask == 0) & (df_feat.Fixate_in_both_conditions == 0)].shape)

(25, 19)
(300, 19)
